In [30]:
from langchain_groq import ChatGroq


In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='',     #your API key
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("Cold email is...")
print(response.content)

Cold email is a form of email marketing where you send an email to someone you don't know, without prior contact or consent, in an attempt to establish a connection, generate interest, or initiate a business relationship. The goal of a cold email is often to introduce a product, service, or opportunity to a potential customer or partner, with the hope of sparking a conversation or securing a meeting.

Cold emails are typically characterized by:

1. Lack of prior relationship: The sender and recipient have no prior connection or communication history.
2. Unsolicited: The email is sent without the recipient's explicit permission or request.
3. Sales or marketing intent: The primary purpose of the email is to promote a product, service, or idea.

Effective cold emails should be:

1. Personalized: Addressed to a specific person, with a clear understanding of their needs and interests.
2. Relevant: The content should be relevant to the recipient's industry, role, or challenges.
3. Concise: 

In [32]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.walmart.com/us/jobs/WD1986214-senior-data-scientist")
page_data = loader.load().pop().page_content
print(page_data)

Senior, Data Scientist | Walmart Careers             















  

    Careers    All Career Areas  Back  Stores & Clubs  Sam’s Club Jobs  Sam’s Club Management Jobs  Walmart Management Jobs  Walmart Store Jobs   Distribution, Fulfillment, & Drivers  Distribution and Fulfillment Centers  Drivers & Transportation  Fleet Maintenance  Flex Associate  Manufacturing   Healthcare  Corporate Healthcare Roles  Optical  Pharmacy   Technology  Cybersecurity  Data Science and Analytics  Information Technology  Project and Program Management - Technology  Software Development and Engineering  UX Design  View all Technology   Corporate  Accounting and Finance  Administrative and Support Services  Asset Protection  Aviation and Travel  Business Operations  Corporate Affairs and Communications  Customer Service and Call Center  Data Analytics and Business Intelligence  Engineering  Ethics and Compliance  Global Investigations and Security  Human Resources  Installation, Maintenance and Utilitie

In [33]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [34]:
print(res.content)

```json
{
  "role": "Senior, Data Scientist",
  "experience": "3+ years",
  "skills": [
    "NLP",
    "Deep Learning",
    "E-commerce",
    "Python",
    "Pyspark",
    "Data Structures",
    "Machine Learning",
    "Data Science"
  ],
  "description": "Drive Digital Landscape modules through application of ML capabilities and enhance with data-derived insights across the wide range of retail divisions. Develop advanced statistical models, machine learning algorithms and computational algorithms based on business initiatives."
}
```


In [35]:
from langchain_core.output_parsers import JsonOutputParser


In [36]:
json_parser = JsonOutputParser()
json_response = json_parser.parse(res.content)

In [37]:
json_response

{'role': 'Senior, Data Scientist',
 'experience': '3+ years',
 'skills': ['NLP',
  'Deep Learning',
  'E-commerce',
  'Python',
  'Pyspark',
  'Data Structures',
  'Machine Learning',
  'Data Science'],
 'description': 'Drive Digital Landscape modules through application of ML capabilities and enhance with data-derived insights across the wide range of retail divisions. Develop advanced statistical models, machine learning algorithms and computational algorithms based on business initiatives.'}

In [38]:
type(json_response)

dict

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv("resume_details.csv")
df

,Name,Email,Phone,Skills,Experience,Education,Projects
0,Abdullah Ahmad,abdullahahmaddd789@gmail.om,+92 311-4739994,"Python, C/C++, SQL, JavaScript, HTML/CSS, Soli...",Volunteer Experience at STACKS-Pakistan FAST C...,Bachelor of Science in Computer Science from F...,Text Restore - Developed a React-based website...


In [48]:
import uuid
import chromadb
import pandas as pd

# 🔹 Load Resume Data
df = pd.read_csv("resume_details.csv")  # Ensure this CSV contains Name, Skills, and Projects

# 🔹 Initialize ChromaDB Client (Persistent Storage)
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")

# 🔹 Insert Data into ChromaDB
if not collection.count():
    for _, row in df.iterrows():
        candidate_name = row.get("Name", "").strip()  # ✅ Ensure Name is fetched
        tech_stack = row.get("Skills", "").strip()  
        project_links = row.get("Projects", "").strip()

        if candidate_name and tech_stack:  # ✅ Ensure Name & Skills exist
            collection.add(
                documents=[tech_stack],  
                metadatas={
                    "name": candidate_name,  # ✅ Store Name properly
                    "links": project_links
                },  
                ids=[str(uuid.uuid4())]
            )

print("✅ Resume data successfully added to ChromaDB!")


✅ Resume data successfully added to ChromaDB!


In [49]:
# 🔹 Fetch the most relevant resume details
resume_data = collection.get(include=["documents", "metadatas"])

if resume_data["documents"]:
    Name = resume_data["metadatas"][0].get("name", "Unknown Candidate")  # ✅ Fetch Name
    skills = resume_data["documents"][0]  # ✅ Fetch Skills as stored
    projects = resume_data["metadatas"][0].get("links", "No Projects Found")  # ✅ Fetch Projects
else:
    Name = "Unknown Candidate"
    skills = "No Skills Found"
    projects = "No Projects Found"

print(f"✅ Retrieved from ChromaDB: Name: {Name}, Skills: {skills}, Projects: {projects}")  # Debugging output


✅ Retrieved from ChromaDB: Name: Unknown Candidate, Skills: Python, C/C++, SQL, JavaScript, HTML/CSS, Solidity, Golang, Assembly, React, Node.js, Flask, Tensorflow, WordPress, Git, Docker, VS Code, Visual Studio, PyCharm, Pandas, NumPy, Matplotlib, Projects: Text Restore - Developed a React-based website. Doing research and developing a super-resolution pipeline to enhance degraded Urdu text images. Created a custom dataset by collecting Urdu manuscripts and applying artificial degradation techniques. Training, Fine-tuning and comparison of state-of-the-art models (SRCNN, ESRGAN, Real-ESRGAN) with PSNR and SSIM metrics for evaluation
Bitcoin Price Predection - Preprocessed Bitcoin data using Pandas, focusing on handling missing values, scaling, and feature engineering for time-series analysis. Planned to develop models predicting daily high, daily low, weekly, and monthly prices for enhanced financial insights
EcoSort - Developed an image classification system to sort waste into catego

In [43]:
# job = json_response

# # Convert skills list to a proper query string
# skills_query = ", ".join(job.get('skills', []))  

# # Query ChromaDB for related projects
# links = collection.query(query_texts=[skills_query], n_results=2).get('metadatas', [])

# print("🔗 Retrieved Project Links:", links)  # ✅ Debugging output


In [44]:
from sentence_transformers import SentenceTransformer
job = json_response
# 🔹 Load a pre-trained model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embedding(text):
    """Convert text into a numerical embedding vector."""
    return embedding_model.encode(text).tolist()

# 🔹 Convert job skills to an embedding
job_embedding = get_embedding(", ".join(job.get('skills', [])))

# 🔹 Query ChromaDB with an embedding search
results = collection.query(query_embeddings=[job_embedding], n_results=2)
links = results.get('metadatas', [])

print("🔗 Retrieved Project Links:", links)


Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


🔗 Retrieved Project Links: [[{'links': 'Text Restore - Developed a React-based website. Doing research and developing a super-resolution pipeline to enhance degraded Urdu text images. Created a custom dataset by collecting Urdu manuscripts and applying artificial degradation techniques. Training, Fine-tuning and comparison of state-of-the-art models (SRCNN, ESRGAN, Real-ESRGAN) with PSNR and SSIM metrics for evaluation\nBitcoin Price Predection - Preprocessed Bitcoin data using Pandas, focusing on handling missing values, scaling, and feature engineering for time-series analysis. Planned to develop models predicting daily high, daily low, weekly, and monthly prices for enhanced financial insights\nEcoSort - Developed an image classification system to sort waste into categories for improved recycling. Built and trained a deep learning model using TensorFlow for accurate waste classification. Deployed a full-stack web application with a Flask backend and a React-based frontend for intera

In [45]:
from langchain.prompts import PromptTemplate

In [47]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are {candidate_name}, a highly skilled professional with expertise in {skills}. 
    You are applying for the above job. Your job is to write a **cold email** to the recruiter, highlighting how your experience 
    and skills align with their needs.
    
    Also, mention the most relevant projects from the following list to showcase your experience: {link_list}

    Do not provide a preamble. Keep the email **professional, concise, and personalized**.
    
    ### EMAIL (NO PREAMBLE):
    
    """
)

# ✅ Combine Prompt & LLM
chain_email = prompt_email | llm  

# 🔹 Ensure ALL required variables are passed
res = chain_email.invoke({
    "candidate_name": Name,  # ✅ Fetched from ChromaDB
    "skills": skills,  # ✅ Fetched from ChromaDB
    "job_description": str(job),
    "link_list": projects  # ✅ Use retrieved projects
})

print(res.content)


Subject: Application for Senior Data Scientist Role

Dear Hiring Manager,

I am excited to apply for the Senior Data Scientist position at your esteemed organization. With over 3 years of experience in developing and deploying machine learning models, I am confident that my skills and expertise align with your requirements.

As a seasoned data scientist, I possess a strong foundation in Python, with expertise in libraries such as Pandas, NumPy, and Matplotlib. My experience with deep learning frameworks, including TensorFlow, has enabled me to develop and train complex models for various applications. I am also proficient in natural language processing (NLP) and have worked on projects involving text analysis and processing.

I would like to highlight a few projects that demonstrate my relevance for this role. In my Text Restore project, I developed a super-resolution pipeline to enhance degraded Urdu text images using state-of-the-art models such as SRCNN, ESRGAN, and Real-ESRGAN. Thi